In [49]:
import random 
import math
import secrets
from decimal import *

Returns the GCD of 2 numbers

In [3]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

Rabin-Miller Primality Test <br>
Check if number is prime, p is the number to be tested and s is the number of rounds <br>
S should ensure probabilitiy of p not being prime is < 2^(-80)

In [6]:
def RMPT(p, s):
    q = p - 1
    k = 0 
    
    # if newP % 2 have remainder then stop
    while q % 2 != 1: 
        q //= 2 
        k += 1 

    for i in range(s):
        #Generate random a 
        a = random.randrange(2, p-2)

        while gcd(a,p) != 1:
            a = random.randrange(2, p-2)
            
        z = pow(a, q, p) # z = a^q mod p

        # checking a^q != 1 mod p and a^q != -1 mod p 
        if z != 1 and z != p-1:
            for j in range(1, k):
                z = pow(z, 2, p)
                if z == 1:
                    return False

            if z != p-1:
                return False
    return True

In [52]:
# Check the result 
print(RMPT(58536828032471131348456426324506659436967195846753702250333704608966791528534067798278728747981548972696886762937373961684727824495513261968904996453134494716095278694362125703914706608360435446792042241177272588181082115012537677828367957226772929218486970585153142173891310402614698182537457744036891034121, 2))

True


AKS Primality Test TODO

Convert a string to ascii

In [14]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [19]:
print(convert_to_ascii('Hello'))

72101108108111


Convert ascii back to String

In [15]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

Generate decryption key using public key e and 2 prime p and q

In [48]:
def RSA_generate_d(e, p, q):
    N = p * q
    phiN = (p-1)*(q-1)
    assert gcd(e, phiN) == 1, "GCD of e and Phi N must be 1"
    d = pow(e, -1, phiN)

    return d


Encrypt with public key e (TODO need to add if plaintext to encrypt is more than N)

In [54]:
def RSA_encrypt(ascii_message, e, N):

    ascii_message = str(ascii_message)
    
    if len(ascii_message) <= N:
        return pow(int(ascii_message), e, N) 

    # Split into chunks
    chunk_size = math.ceil(len(ascii_message) / N)
    chunks = [ascii_message[i:i+chunk_size] for i in range(0, len(ascii_message), chunk_size)]
    
    encrypted_chunks = []
    for chunk in chunks:
        encrypted_chunks.append(pow(int(chunk), e, N))
    
    return encrypted_chunks

In [55]:
Test = convert_to_ascii('This is a very long message that needs to be split into smaller equal chunks')
encryptTest = RSA_encrypt(Test, e, 20)


Decrypt using private key d

In [56]:
def RSA_decrypt(encrypted_chunks, d, N):

    decrypted_chunks = []
    for chunk in encrypted_chunks:
        decrypted_chunks.append(pow(chunk, d, N))
    
    # Reconstruct original message    
    ascii_msg = ""
    for chunk in decrypted_chunks:
        ascii_msg += str(chunk)
    
    return ascii_msg

In [58]:
dTest = RSA_decrypt(encryptTest, d, 20)
print(convert_to_string(dTest))

o
ooos


Generate Primes p and q 

In [47]:
def generate_prime(bits, rounds):
    while(True):
        num = secrets.randbits(bits) # using CSRPNG to generate

        # if number is even make it odd 
        if num % 2 == 0:
            num += 1

        elif num % 3 == 0:
            continue 

        elif num % 5 == 0:
            continue

        elif num % 7 == 0:
            continue

        elif num % 11 == 0: 
            continue

        elif RMPT(num, rounds):
            return num


## RSA 1024 

Encrypt message using RSA 1024 
<br><br>
Generate primes p and q <br>
Encryption exponent e = 3 <br>

<b> Finding decryption exponent </b>

In [35]:
e = 3
phi_test = False

while (phi_test == False):
    p = generate_prime(1024, 3)
    q = generate_prime(1024, 3)

    phiN = (p-1) * (q-1)

    if gcd(e, phiN) == 1:
        phi_test = True

print('p is: ', p)
print('q is: ', q)
d = RSA_generate_d(e, p, q)
N = p * q

print('N is: ', N)

p is:  71297216456396582361426341562446439213900335393707388597368164393257417615437893621251165743944699167344562499412933812629153631106423965638951873017925997131392745728341359012644079378170415902877812115772453652160646309356970130849350418487389071470724067110280436601999991107950398944432228084586863257021
q is:  114533163162290324551442091975204729849625947986014386615138024839592173951172473282459601234575452930681813800354159329759274205390365704832876561369804609485010470714341169554183127799256722731118417775450870860582484197931521304617210446628979589450882549531842390108520996185013274920675765373622696341581
N is:  816589572541760055856718750931824319158853190457898688777571510372639368955109324069379643419381143905004767519433166102544676683177803773074909779145355940166034447755093686872936595025034308827701662396871986326904947790117048000362164242118776088303130028976763787019092872755409605683679277367349800049438116377722434181380677766260815140410631181826342823

Preparing message for encryption, converting it to ascii

In [55]:
message = "Your Auntie"

print('Initial plaintext: ', message)
ascii_msg = convert_to_ascii(message)
print('Converted ascii msg: ', ascii_msg)
ascii_msg.bit_length()

Initial plaintext:  Your Auntie
Converted ascii msg:  89111117114032065117110116105101


107

Begin encryption

In [59]:
encrypted_message = RSA_encrypt(ascii_msg, e, N)
print('Encrypted msg in ascii: ', encrypted_message)

print('Encrypted msg in string: ', convert_to_string(encrypted_message))


Encrypted msg in ascii:  707612774005187645873348056538847247660687871307776743683103457680673942077710860630928737345301
Encrypted msg in string:  ˃ɤ̆»ʅͩŜ8Ț͏÷ʔʯͧĳ̈˧ʫgǉʨʡήMˆ͜ɶΠˡřĭ


Decryption with d

In [61]:
decrypted_message = RSA_decrypt(encrypted_message, d, N)

print('Decrypted msg in ascii: ', decrypted_message)
print('Decrypted msg in string: ', convert_to_string(decrypted_message))

Decrypted msg in ascii:  89111117114032065117110116105101
Decrypted msg in string:  Your Auntie
